# workflow.ipynb

## qrcode_generator.ipynb

Generates small QRcode labels containing sequrntial serial numbers. 
Each label is placed at the center of a sticky trap to uniquely identify the trap.
The labelalso acts as a target which facilitates autofocus.
**get_image.py** can read the QRcode.
```
Usage: papermill [OPTIONS] NOTEBOOK_PATH [OUTPUT_PATH]

Parameters inferred for notebook '/home/aubrey/Desktop/libcamera/qrcode_generator.ipynb':
  FONT_SIZE: Unknown type (default 4)
                                  points
  FONT: Unknown type (default 'helvetica')
  YMIN: Unknown type (default 1)
                                  inches
  LABEL_WIDTH: Unknown type (default 0.5)
                                  inches
  LABEL_HEIGHT: Unknown type (default 0.5)
                                  inches
  XMIN: Unknown type (default 0.5)
                                  inches
  NROWS: Unknown type (default 10)
  NCOLS: Unknown type (default 10)
  PRINT: Unknown type (default False)
  MIN_SN: Unknown type (default 1)
  MAX_SN: Unknown type (default 100)
  PAGE_HEADER: Unknown type (default 'serial numbers generated by qrcode_generator.ipynb')
  PDF_FILENAME: Unknown type (default 'qrcodes.pdf')
```

## get_image.ipynb
Gets a single sticky trap image from Raspberry Pi 16MP autofocus camera.
```
Parameters inferred for notebook '/home/aubrey/Desktop/libcamera/get_image.ipynb':
  RPI_IP: Unknown type (default "192.168.1.102")
  RPI_USERNAME: Unknown type (default "pi")
  RPI_PASSWORD: Unknown type (default "canada12")
  TRAP_ID: Unknown type (default 999)
  USE_QRCODE: Unknown type (default True)
  OUTPUT_DIR: Unknown type (default './tinian_cycads')
```

## crop_square_centered_on_qrcode.ipynb
Crops a single sticky trap image using the QRcode as a center point.
```
Parameters inferred for notebook '/home/aubrey/Desktop/libcamera/crop_square_centered_on_qrcode.ipynb':
  INPUT_DIR: Unknown type (default './tinian_cycads')
  OUTPUT_DIR: Unknown type (default './tinian_cycads_cropped')
  CROP_WIDTH: Unknown type (default 2500)
```

## sticky_trap_image_analysis.ipynb
Detects insects and other particles on a sticky trap image. Each insect/particle is saved in OUTPUTPATH.
```
Parameters inferred for notebook '/home/aubrey/Desktop/counting-for-entomologists/sticky_trap_image_analysis.ipynb':
  IMAGEPATH: Unknown type (default '/home/aubrey/Desktop/my_sticky_trap.jpg')
  OUTPUTPATH: Unknown type (default 'my_sticky_trap')
  CANNY: Unknown type (default 45)
```

## clustimage
```
Parameters inferred for notebook '/home/aubrey/Desktop/tinian_cycad_scale/code/my_clustimage.ipynb':
  FILE_LIST_GLOB: Unknown type (default '/home/aubrey/Desktop/libcamera/tinian_cycads_cropped/40/0*a.png')
```

# TODO
* remove rotation in sticky_trap_analysis.ipynb
* rename sticky_trap_analysis.ipynb to extract_bug_images.ipynb
* move Jupyter notebooks into sticky_trap_imaging/code

In [2]:
import os

In [3]:
os.getcwd()

'/home/aubrey/Desktop/tinian_cycad_scale/code'